In [4]:
import os
from estnltk import Text
from estnltk_neural.taggers import StanzaSyntaxTagger
import pandas as pd
import copy
import numpy as np
import sklearn
import sqlite3
from estnltk.converters import text_to_json
import json

Alustuseks fraasimustrid, mis peaksid olema ebakorrektsed. Fraasimustrid on tekstifailis, igal real üks muster. Esituskuju on järgnev: id-ner/timex/other-ülemuse_id-deprel-POS eraldatuna tühikutega. Eri sõnade info on eraldatud komadega. Juhul, kui POS on mitmene, esitatakse variandid püstkriipsuga eraldatult. Indekseid mustrites ei esitata, sest fraasisiseste otsingut korral võivad tekkida ebakõlad. Liikmete järjestus vastab järjestusele tekstis.



In [1]:
# patterns are kept in a list of strings
patterns = None

with open('incorrect_tree_patterns.txt', 'r') as f:
    patterns = [line.rstrip() for line in f]

for pattern in patterns:
    print(pattern)

1 PER 0 root H,2 PER 1 flat H,3 OTHER 1 flat Y
1 PER 0 root H,2 PER 1 flat S,3 OTHER 1 flat S
1 LOC 0 root H,2 OTHER 1 nmod S
1 ORG 0 root Y,2 OTHER 1 nmod S
1 LOC 2 nmod H,2 OTHER 0 root P|N
1 LOC 0 root H,2 OTHER 1 punct Z,3 OTHER 1 nmod S
1 LOC 2 nmod H,2 OTHER 0 root S,3 OTHER 2 nmod H
1 OTHER 2 punct Z,2 LOC 0 root H,3 OTHER 2 punct Z
1 OTHER 2 punct Z,2 PER 0 root S,3 OTHER 2 punct Z
1 OTHER 2 punct Z,2 ORG 0 root Y,3 OTHER 2 punct Z
1 LOC 3 nmod H,2 OTHER 3 amod P|O,3 OTHER 0 root S
1 OTHER 3 punct Z,2 LOC 3 nmod H,3 OTHER 0 root S
1 PER 3 punct Z,2 PER 3 nmod S,3 OTHER 0 root S
1 OTHER 3 punct Z,2 ORG 3 nmod Y,3 OTHER 0 root S


In [2]:
patterns_simplified = []

for pattern in patterns:
    pieces = [wrd.split() for wrd in pattern.split(',')]
    for piece in pieces:
        piece.pop(1)

    for i in range(len(pieces)):
        pieces[i] = ' '.join(pieces[i])
    patterns_simplified.append(','.join(pieces))
    
for pattern in patterns_simplified:
    print(pattern)

1 0 root H,2 1 flat H,3 1 flat Y
1 0 root H,2 1 flat S,3 1 flat S
1 0 root H,2 1 nmod S
1 0 root Y,2 1 nmod S
1 2 nmod H,2 0 root P|N
1 0 root H,2 1 punct Z,3 1 nmod S
1 2 nmod H,2 0 root S,3 2 nmod H
1 2 punct Z,2 0 root H,3 2 punct Z
1 2 punct Z,2 0 root S,3 2 punct Z
1 2 punct Z,2 0 root Y,3 2 punct Z
1 3 nmod H,2 3 amod P|O,3 0 root S
1 3 punct Z,2 3 nmod H,3 0 root S
1 3 punct Z,2 3 nmod S,3 0 root S
1 3 punct Z,2 3 nmod Y,3 0 root S


In [5]:
df = pd.read_pickle('phrase_examples_filtered_sub10000/non_atomic_phrases.pkl')

In [6]:
df

,phrase,phrase_length,document_creation_time,sentence_id,document_id,sentence_startend,subcorpus,phrase_type,phrase_start_end,has_ner_netity,...,pos_sequence,graph,graph_code,graph_code_pos,graph_code_pos_ner_timex,pos_sequence_verb_info,graph_verb_info,graph_code_verb_info,graph_code_pos_verb_info,graph_code_pos_ner_timex_verb_info
0,Text(text='neist teede äärtesse'),3,2024-03-10T00:43,5858921,312319,"(0, 226)",aja_EPL,obl_phrase,"(84, 104)",0,...,P-S-S,"(1, 2, 3, 0)","((0, 3, root),(2, 1, det),(3, 2, nmod))","(S-P-S,(0, 3, root),(2, 1, det),(3, 2, nmod))","(0-0-0,S-P-S,(0, 3, root),(2, 1, det),(3, 2, nmod))",P-S-S,"(1, 2, 3, 0)","((0, 3, root),(2, 1, det),(3, 2, nmod))","(S-P-S,(0, 3, root),(2, 1, det),(3, 2, nmod))","(0-0-0,S-P-S,(0, 3, root),(2, 1, det),(3, 2, nmod))"
1,Text(text='Tartu ülikooli ajaloo osakonna üldajaloo õppetooli vanemlaborant Sirje Tamul'),9,2024-03-10T00:43,11085184,487500,"(227, 375)",aja_pm,nsubj_cop_phrase,"(0, 76)",1,...,H-S-S-S-S-S-S-H-H,"(1, 2, 3, 4, 5, 6, 7, 0, 8, 9)","((0, 7, root),(2, 1, nmod),(3, 2, nmod),(4, 3, nmod),(5, 4, nmod),(6, 5, nmod),(7, 6, nmod),(7, 8, appos),(8, 9, flat))","(S-H-S-S-S-S-S-H-H,(0, 7, root),(2, 1, nmod),(3, 2, nmod),(4, 3, nmod),(5, 4, nmod),(6, 5, nmod),(7, 6, nmod),(7, 8, appos),(8, 9, flat))","(0-ner_ORG-ner_ORG-0-0-0-0-0-0,S-H-S-S-S-S-S-H-H,(0, 7, root),(2, 1, nmod),(3, 2, nmod),(4, 3, nmod),(5, 4, nmod),(6, 5, nmod),(7, 6, nmod),(7, 8, appos),(8, 9, flat))",H-S-S-S-S-S-S-H-H,"(1, 2, 3, 4, 5, 6, 7, 0, 8, 9)","((0, 7, root),(2, 1, nmod),(3, 2, nmod),(4, 3, nmod),(5, 4, nmod),(6, 5, nmod),(7, 6, nmod),(7, 8, appos),(8, 9, flat))","(S-H-S-S-S-S-S-H-H,(0, 7, root),(2, 1, nmod),(3, 2, nmod),(4, 3, nmod),(5, 4, nmod),(6, 5, nmod),(7, 6, nmod),(7, 8, appos),(8, 9, flat))","(0-ner_ORG-ner_ORG-0-0-0-0-0-0,S-H-S-S-S-S-S-H-H,(0, 7, root),(2, 1, nmod),(3, 2, nmod),(4, 3, nmod),(5, 4, nmod),(6, 5, nmod),(7, 6, nmod),(7, 8, appos),(8, 9, flat))"
2,Text(text='Tartu ülikooli ajaloo'),3,2024-03-10T00:43,11085184,487500,"(227, 375)",aja_pm,nmod_phrase,"(0, 21)",1,...,H-S-S,"(1, 2, 3, 0)","((0, 3, root),(2, 1, nmod),(3, 2, nmod))","(S-H-S,(0, 3, root),(2, 1, nmod),(3, 2, nmod))","(0-ner_ORG-ner_ORG,S-H-S,(0, 3, root),(2, 1, nmod),(3, 2, nmod))",H-S-S,"(1, 2, 3, 0)","((0, 3, root),(2, 1, nmod),(3, 2, nmod))","(S-H-S,(0, 3, root),(2, 1, nmod),(3, 2, nmod))","(0-ner_ORG-ner_ORG,S-H-S,(0, 3, root),(2, 1, nmod),(3, 2, nmod))"
3,Text(text='Tartu ülikooli ajaloo osakonna'),4,2024-03-10T00:43,11085184,487500,"(227, 375)",aja_pm,nmod_phrase,"(0, 30)",1,...,H-S-S-S,"(1, 2, 3, 4, 0)","((0, 4, root),(2, 1, nmod),(3, 2, nmod),(4, 3, nmod))","(S-H-S-S,(0, 4, root),(2, 1, nmod),(3, 2, nmod),(4, 3, nmod))","(0-ner_ORG-ner_ORG-0,S-H-S-S,(0, 4, root),(2, 1, nmod),(3, 2, nmod),(4, 3, nmod))",H-S-S-S,"(1, 2, 3, 4, 0)","((0, 4, root),(2, 1, nmod),(3, 2, nmod),(4, 3, nmod))","(S-H-S-S,(0, 4, root),(2, 1, nmod),(3, 2, nmod),(4, 3, nmod))","(0-ner_ORG-ner_ORG-0,S-H-S-S,(0, 4, root),(2, 1, nmod),(3, 2, nmod),(4, 3, nmod))"
4,Text(text='Tartu ülikooli ajaloo osakonna üldajaloo õppetooli'),6,2024-03-10T00:43,11085184,487500,"(227, 375)",aja_pm,nmod_phrase,"(0, 50)",1,...,H-S-S-S-S-S,"(1, 2, 3, 4, 5, 6, 0)","((0, 6, root),(2, 1, nmod),(3, 2, nmod),(4, 3, nmod),(5, 4, nmod),(6, 5, nmod))","(S-H-S-S-S-S,(0, 6, root),(2, 1, nmod),(3, 2, nmod),(4, 3, nmod),(5, 4, nmod),(6, 5, nmod))","(0-ner_ORG-ner_ORG-0-0-0,S-H-S-S-S-S,(0, 6, root),(2, 1, nmod),(3, 2, nmod),(4, 3, nmod),(5, 4, nmod),(6, 5, nmod))",H-S-S-S-S-S,"(1, 2, 3, 4, 5, 6, 0)","((0, 6, root),(2, 1, nmod),(3, 2, nmod),(4, 3, nmod),(5, 4, nmod),(6, 5, nmod))","(S-H-S-S-S-S,(0, 6, root),(2, 1, nmod),(3, 2, nmod),(4, 3, nmod),(5, 4, nmod),(6, 5, nmod))","(0-ner_ORG-ner_ORG-0-0-0,S-H-S-S-S-S,(0, 6, root),(2, 1, nmod),(3, 2, nmod),(4, 3, nmod),(5, 4, nmod),(6, 5, nmod))"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3051,Text(text='konkureeriva kaabeltelevisooni firma'),3,2024-03-10T01:35,1526710,61973,"(522, 718)",aja_

In [7]:
test = None
for idx, row in df.iterrows():
    test = row['phrase']
    break

In [8]:
test.stanza_syntax

Layer(name='stanza_syntax', attributes=('id', 'lemma', 'upostag', 'xpostag', 'feats', 'head', 'deprel', 'deps', 'misc', 'parent_span', 'children'), spans=SL[Span('neist', [{'id': 1, 'lemma': 'see', 'upostag': 'P', 'xpostag': 'P', 'feats': OrderedDict([('pl', 'pl'), ('el', 'el')]), 'head': 2, 'deprel': 'det', 'deps': '_', 'misc': '_', 'parent_span': <class 'estnltk_core.layer.span.Span'>, 'children': ()}]),
Span('teede', [{'id': 2, 'lemma': 'tee', 'upostag': 'S', 'xpostag': 'S', 'feats': OrderedDict([('pl', 'pl'), ('g', 'g')]), 'head': 3, 'deprel': 'nmod', 'deps': '_', 'misc': '_', 'parent_span': <class 'estnltk_core.layer.span.Span'>, 'children': <class 'tuple'>}]),
Span('äärtesse', [{'id': 3, 'lemma': 'äär', 'upostag': 'S', 'xpostag': 'S', 'feats': OrderedDict([('pl', 'pl'), ('ill', 'ill')]), 'head': 0, 'deprel': 'root', 'deps': '_', 'misc': '_', 'parent_span': None, 'children': <class 'tuple'>}])])

In [9]:
def get_ner_timex(text_obj, stanza_word):
    ner = None
    nertag = None
    if len(text_obj.ner) > 0:
        word = text_obj.words.get(stanza_word)
        for n in text_obj.ner:
            for part in n:
                if part==word:
                    ner=word
                    nertag=n.nertag
    timex = text_obj.timexes.get(stanza_word)
    if ner:
        return nertag
    if timex:
        return 'TIMEX'
    return 'OTHER'

In [10]:
def get_unique_POS(word):
    #pos_list = word.morph_analysis['partofspeech']
    infinite_verb_forms = ['da', 'des', 'ma', 'maks', 'mas', 'mast', 'mata', 'nud', 'tav', 'tud', 'v']
    # if POS is ambiguous, only unique tags are kept, e.g. ['V', 'A', 'A'] -> ['V', 'A']
    pos_list = []
    for i in range(len(word.morph_analysis['partofspeech'])):
        if word.morph_analysis['partofspeech'][i] == 'V':
            if word.morph_analysis['form'][i] in infinite_verb_forms:
                pos_list.append('V_inf')
            elif word.form[i] == 'neg':
                pos_list.append('V_neg')
            else:
                pos_list.append('V_fin')
        else:
            pos_list.append(word.morph_analysis['partofspeech'][i])
    
    if len(pos_list) > 1:
        char_unique = [char for indx, char in enumerate(pos_list) if char not in pos_list[:indx]]
        if len(char_unique) < 2:
            return char_unique[0]
        return '|'.join(char_unique)
    return pos_list[0]

In [11]:
test2 = None
for idx, row in df.iterrows():
    if idx == 0:
        test2 = row['phrase']
        break

In [12]:
test2.stanza_syntax

Layer(name='stanza_syntax', attributes=('id', 'lemma', 'upostag', 'xpostag', 'feats', 'head', 'deprel', 'deps', 'misc', 'parent_span', 'children'), spans=SL[Span('neist', [{'id': 1, 'lemma': 'see', 'upostag': 'P', 'xpostag': 'P', 'feats': OrderedDict([('pl', 'pl'), ('el', 'el')]), 'head': 2, 'deprel': 'det', 'deps': '_', 'misc': '_', 'parent_span': <class 'estnltk_core.layer.span.Span'>, 'children': ()}]),
Span('teede', [{'id': 2, 'lemma': 'tee', 'upostag': 'S', 'xpostag': 'S', 'feats': OrderedDict([('pl', 'pl'), ('g', 'g')]), 'head': 3, 'deprel': 'nmod', 'deps': '_', 'misc': '_', 'parent_span': <class 'estnltk_core.layer.span.Span'>, 'children': <class 'tuple'>}]),
Span('äärtesse', [{'id': 3, 'lemma': 'äär', 'upostag': 'S', 'xpostag': 'S', 'feats': OrderedDict([('pl', 'pl'), ('ill', 'ill')]), 'head': 0, 'deprel': 'root', 'deps': '_', 'misc': '_', 'parent_span': None, 'children': <class 'tuple'>}])])

In [13]:
# getting patterns without ner/timex/other
pattern_matches_found = []

for i in range(len(test2.stanza_syntax) - 1):
    pattern_members = []
    ids = []

    pattern_members.append(test2.stanza_syntax[i])
    #ner_timex = get_ner_timex(test2, test2.stanza_syntax[i])
    ids.append([test2.stanza_syntax[i].id, test2.stanza_syntax[i].head])

    for j in range(i + 1, len(test2.stanza_syntax)):
        pattern_found = []
        current_members = []
        for k in range(len(pattern_members)):
            #print(member)
            #print(test2.stanza_syntax[j])
            if test2.stanza_syntax[j] in pattern_members[k].children or pattern_members[k] in test2.stanza_syntax[j].children or test2.stanza_syntax[j].parent != None and test2.stanza_syntax[j].parent == pattern_members[k].parent:
                current_members.append(pattern_members[k])
                current_members.append(test2.stanza_syntax[j])
                pattern_members.append(test2.stanza_syntax[j])
                #ner_timex = get_ner_timex(test2, test2.stanza_syntax[j])
                ids.append([test2.stanza_syntax[j].id, test2.stanza_syntax[j].head])
        
        ids_for_pattern = copy.deepcopy(ids)
        for l in range(len(ids_for_pattern)):
            temp = ids_for_pattern[l][0]
            ids_for_pattern[l][0] = l+1
            for m in range(len(ids)):
                if ids[m][1] == temp:
                    ids_for_pattern[m][1] = ids_for_pattern[l][0]
            
        word_ids = [word_id[0] for word_id in ids_for_pattern]
        for l in range(len(ids_for_pattern)):
            if ids_for_pattern[l][0] == ids_for_pattern[l][1]:
                ids_for_pattern[l][1] = 0
            elif ids_for_pattern[l][1] not in word_ids:
                ids_for_pattern[l][1] = 0
            
        for l in range(len(pattern_members)):
            deprel = pattern_members[l].deprel
            if ids_for_pattern[l][1] == 0 and deprel != 'root':
                deprel = 'root'
            pattern_found.append([str(ids_for_pattern[l][0]), str(ids_for_pattern[l][1]), deprel, get_unique_POS(pattern_members[l])])
        
        sub_pattern = [" ".join(word_info) for word_info in pattern_found]
        print(sub_pattern)
        print(",".join(sub_pattern))
        print(",".join(sub_pattern) in patterns_simplified)
        if ",".join(sub_pattern) in patterns_simplified:
            pattern_matches_found.append([",".join(sub_pattern), " ".join([member.text for member in pattern_members]), [member for member in pattern_members]])

        if j == len(test2.stanza_syntax) - 1:
            pattern_members = []
            ids = []
            break

print(pattern_matches_found)

['1 2 det P', '2 0 root S']
1 2 det P,2 0 root S
False
['1 2 det P', '2 3 nmod S', '3 0 root S']
1 2 det P,2 3 nmod S,3 0 root S
False
['1 2 nmod S', '2 0 root S']
1 2 nmod S,2 0 root S
False
[]


In [14]:
# getting patterns with ner/timex/other
pattern_matches_found = []

for i in range(len(test2.stanza_syntax) - 1):
    pattern_members = []
    ner_timex = []
    ids = []

    pattern_members.append(test2.stanza_syntax[i])
    ner_timex.append(get_ner_timex(test2, test2.stanza_syntax[i]))
    ids.append([test2.stanza_syntax[i].id, test2.stanza_syntax[i].head])

    for j in range(i + 1, len(test2.stanza_syntax)):
        pattern_found = []
        current_members = []
        for k in range(len(pattern_members)):
            #print(member)
            #print(test2.stanza_syntax[j])
            if test2.stanza_syntax[j] in pattern_members[k].children or pattern_members[k] in test2.stanza_syntax[j].children or test2.stanza_syntax[j].parent != None and test2.stanza_syntax[j].parent == pattern_members[k].parent:
                current_members.append(pattern_members[k])
                current_members.append(test2.stanza_syntax[j])
                pattern_members.append(test2.stanza_syntax[j])
                ner_timex.append(get_ner_timex(test2, test2.stanza_syntax[j]))
                ids.append([test2.stanza_syntax[j].id, test2.stanza_syntax[j].head])
        
        ids_for_pattern = copy.deepcopy(ids)
        for l in range(len(ids_for_pattern)):
            temp = ids_for_pattern[l][0]
            ids_for_pattern[l][0] = l+1
            for m in range(len(ids)):
                if ids[m][1] == temp:
                    ids_for_pattern[m][1] = ids_for_pattern[l][0]
            
        word_ids = [word_id[0] for word_id in ids_for_pattern]
        for l in range(len(ids_for_pattern)):
            if ids_for_pattern[l][0] == ids_for_pattern[l][1]:
                ids_for_pattern[l][1] = 0
            elif ids_for_pattern[l][1] not in word_ids:
                ids_for_pattern[l][1] = 0
            
        for l in range(len(pattern_members)):
            deprel = pattern_members[l].deprel
            if ids_for_pattern[l][1] == 0 and deprel != 'root':
                deprel = 'root'
            pattern_found.append([str(ids_for_pattern[l][0]), ner_timex[l], str(ids_for_pattern[l][1]), deprel, get_unique_POS(pattern_members[l])])
        
        sub_pattern = [" ".join(word_info) for word_info in pattern_found]
        print(sub_pattern)
        print(",".join(sub_pattern))
        print(",".join(sub_pattern) in patterns_simplified)
        if ",".join(sub_pattern) in patterns_simplified:
            pattern_matches_found.append(",".join(sub_pattern))

        if j == len(test2.stanza_syntax) - 1:
            pattern_members = []
            ids = []
            break
                

['1 OTHER 2 det P', '2 OTHER 0 root S']
1 OTHER 2 det P,2 OTHER 0 root S
False
['1 OTHER 2 det P', '2 OTHER 3 nmod S', '3 OTHER 0 root S']
1 OTHER 2 det P,2 OTHER 3 nmod S,3 OTHER 0 root S
False
['1 OTHER 2 nmod S', '2 OTHER 0 root S']
1 OTHER 2 nmod S,2 OTHER 0 root S
False


### On more data

In [15]:
def find_patterns(phrase):
    pattern_matches_found = []
    
    for i in range(len(phrase.stanza_syntax) - 1):
        pattern_members = []
        ids = []

        pattern_members.append(phrase.stanza_syntax[i])
        #ner_timex = get_ner_timex(phrase, phrase.stanza_syntax[i])
        ids.append([phrase.stanza_syntax[i].id, phrase.stanza_syntax[i].head])

        for j in range(i + 1, len(phrase.stanza_syntax)):
            pattern_found = []
            for k in range(len(pattern_members)):
                if phrase.stanza_syntax[j] in pattern_members[k].children or pattern_members[k] in phrase.stanza_syntax[j].children or phrase.stanza_syntax[j].parent != None and phrase.stanza_syntax[j].parent == pattern_members[k].parent:
                    pattern_members.append(phrase.stanza_syntax[j])
                    #ner_timex = get_ner_timex(phrase, phrase.stanza_syntax[j])
                    ids.append([phrase.stanza_syntax[j].id, phrase.stanza_syntax[j].head])
        
            ids_for_pattern = copy.deepcopy(ids)
            for l in range(len(ids_for_pattern)):
                temp = ids_for_pattern[l][0]
                ids_for_pattern[l][0] = l+1
                for m in range(len(ids)):
                    if ids[m][1] == temp:
                        ids_for_pattern[m][1] = ids_for_pattern[l][0]
            
            word_ids = [word_id[0] for word_id in ids_for_pattern]
            for l in range(len(ids_for_pattern)):
                if ids_for_pattern[l][0] == ids_for_pattern[l][1]:
                    ids_for_pattern[l][1] = 0
                elif ids_for_pattern[l][1] not in word_ids:
                    ids_for_pattern[l][1] = 0
            
            for l in range(len(pattern_members)):
                deprel = pattern_members[l].deprel
                if ids_for_pattern[l][1] == 0 and deprel != 'root':
                    deprel = 'root'
                pattern_found.append([str(ids_for_pattern[l][0]), str(ids_for_pattern[l][1]), deprel, get_unique_POS(pattern_members[l])])
                    
            sub_pattern = [" ".join(word_info) for word_info in pattern_found]
            if ",".join(sub_pattern) in patterns_simplified:
                pattern_matches_found.append([",".join(sub_pattern), " ".join([member.text for member in pattern_members]), [member for member in pattern_members]])

            if j == len(test2.stanza_syntax) - 1:
                pattern_members = []
                ids = []
                break             
                    
    return pattern_matches_found
                

In [ ]:
def find_patterns_ner_timex(phrase):
pattern_matches_found = []

for i in range(len(test2.stanza_syntax) - 1):
    pattern_members = []
    ner_timex = []
    ids = []

    pattern_members.append(test2.stanza_syntax[i])
    ner_timex.append(get_ner_timex(test2, test2.stanza_syntax[i]))
    ids.append([test2.stanza_syntax[i].id, test2.stanza_syntax[i].head])

    for j in range(i + 1, len(test2.stanza_syntax)):
        pattern_found = []
        current_members = []
        for k in range(len(pattern_members)):
            #print(member)
            #print(test2.stanza_syntax[j])
            if test2.stanza_syntax[j] in pattern_members[k].children or pattern_members[k] in test2.stanza_syntax[j].children or test2.stanza_syntax[j].parent != None and test2.stanza_syntax[j].parent == pattern_members[k].parent:
                current_members.append(pattern_members[k])
                current_members.append(test2.stanza_syntax[j])
                pattern_members.append(test2.stanza_syntax[j])
                ner_timex.append(get_ner_timex(test2, test2.stanza_syntax[j]))
                ids.append([test2.stanza_syntax[j].id, test2.stanza_syntax[j].head])
        
        ids_for_pattern = copy.deepcopy(ids)
        for l in range(len(ids_for_pattern)):
            temp = ids_for_pattern[l][0]
            ids_for_pattern[l][0] = l+1
            for m in range(len(ids)):
                if ids[m][1] == temp:
                    ids_for_pattern[m][1] = ids_for_pattern[l][0]
            
        word_ids = [word_id[0] for word_id in ids_for_pattern]
        for l in range(len(ids_for_pattern)):
            if ids_for_pattern[l][0] == ids_for_pattern[l][1]:
                ids_for_pattern[l][1] = 0
            elif ids_for_pattern[l][1] not in word_ids:
                ids_for_pattern[l][1] = 0
            
        for l in range(len(pattern_members)):
            deprel = pattern_members[l].deprel
            if ids_for_pattern[l][1] == 0 and deprel != 'root':
                deprel = 'root'
            pattern_found.append([str(ids_for_pattern[l][0]), ner_timex[l], str(ids_for_pattern[l][1]), deprel, get_unique_POS(pattern_members[l])])
        
        sub_pattern = [" ".join(word_info) for word_info in pattern_found]
        print(sub_pattern)
        print(",".join(sub_pattern))
        print(",".join(sub_pattern) in patterns_simplified)
        if ",".join(sub_pattern) in patterns_simplified:
            pattern_matches_found.append(",".join(sub_pattern))

        if j == len(test2.stanza_syntax) - 1:
            pattern_members = []
            ids = []
            break

In [17]:
# searching for matches from both atomic and non-atomic phrases
df_non_atomic = pd.read_pickle('phrase_examples_filtered_sub10000/non_atomic_phrases.pkl')
df_atomic = pd.read_pickle('phrase_examples_filtered_sub10000/atomic_phrases.pkl')
concatenated = pd.concat([df_non_atomic, df_atomic])

incorrect_matches = {'pattern': [], 'phrase_text': [], 'phrase_spans': [], 'parent_phrase': []}

for idx, row in concatenated.iterrows():
    matches = find_patterns(row['phrase'])
    if len(matches) > 0:
        for match in matches:
            incorrect_matches['pattern'].append(match[0])
            incorrect_matches['phrase_text'].append(match[1])
            incorrect_matches['phrase_spans'].append(match[2])
            incorrect_matches['parent_phrase'].append(row['phrase'])
            
matches_df = pd.DataFrame(incorrect_matches)

In [18]:
matches_df

,pattern,phrase_text,phrase_spans,parent_phrase
0,"1 0 root Y,2 1 nmod S",MM turniiriks,"[Span('MM', [{'id': 3, 'lemma': 'mm', 'upostag': 'Y', 'xpostag': 'Y', 'feats': OrderedDict([('?', '?')]), 'head': 0, 'deprel': 'root', 'deps': '_', 'misc': '_', 'parent_span': None, 'children': <class 'tuple'>}]), Span('turniiriks', [{'id': 5, 'lemma': 'turniir', 'upostag': 'S', 'xpostag': 'S', 'feats': OrderedDict([('sg', 'sg'), ('tr', 'tr')]), 'head': 3, 'deprel': 'nmod', 'deps': '_', 'misc': '_', 'parent_span': <class 'estnltk_core.layer.span.Span'>, 'children': <class 'tuple'>}])]",Text(text='veel kümneks MM-turniiriks')
1,"1 2 punct Z,2 0 root S,3 2 punct Z",", häbi .","[Span(',', [{'id': 6, 'lemma': ',', 'upostag': 'Z', 'xpostag': 'Z', 'feats': OrderedDict(), 'head': 10, 'deprel': 'punct', 'deps': '_', 'misc': '_', 'parent_span': <class 'estnltk_core.layer.span.Span'>, 'children': ()}]), Span('häbi', [{'id': 10, 'lemma': 'häbi', 'upostag': 'S', 'xpostag': 'S', 'feats': OrderedDict([('sg', 'sg'), ('n', 'n')]), 'head': 2, 'deprel': 'conj', 'deps': '_', 'misc': '_', 'parent_span': <class 'estnltk_core.layer.span.Span'>, 'children': <class 'tuple'>}]), Span('.', [{'id': 11, 'lemma': '.', 'upostag': 'Z', 'xpostag': 'Z', 'feats': OrderedDict(), 'head': 10, 'deprel': 'punct', 'deps': '_', 'misc': '_', 'parent_span': <class 'estnltk_core.layer.span.Span'>, 'children': ()}])]","Text(text='Sajad politseinikud kahe mehe vastu , no on alles häbi . ""')"
2,"1 2 punct Z,2 0 root S,3 2 punct Z",", häbi .","[Span(',', [{'id': 6, 'lemma': ',', 'upostag': 'Z', 'xpostag': 'Z', 'feats': OrderedDict(), 'head': 10, 'deprel': 'punct', 'deps': '_', 'misc': '_', 'parent_span': <class 'estnltk_core.layer.span.Span'>, 'children': ()}]), Span('häbi', [{'id': 10, 'lemma': 'häbi', 'upostag': 'S', 'xpostag': 'S', 'feats': OrderedDict([('sg', 'sg'), ('n', 'n')]), 'head': 2, 'deprel': 'conj', 'deps': '_', 'misc': '_', 'parent_span': <class 'estnltk_core.layer.span.Span'>, 'children': <class 'tuple'>}]), Span('.', [{'id': 11, 'lemma': '.', 'upostag': 'Z', 'xpostag': 'Z', 'feats': OrderedDict(), 'head': 10, 'deprel': 'punct', 'deps': '_', 'misc': '_', 'parent_span': <class 'estnltk_core.layer.span.Span'>, 'children': ()}])]","Text(text='Sajad politseinikud kahe mehe vastu , no on alles häbi . ""')"
3,"1 2 punct Z,2 0 root S,3 2 punct Z",""" osa ""","[Span('""', [{'id': 3, 'lemma': '""', 'upostag': 'Z', 'xpostag': 'Z', 'feats': OrderedDict(), 'head': 5, 'deprel': 'punct', 'deps': '_', 'misc': '_', 'parent_span': <class 'estnltk_core.layer.span.Span'>, 'children': ()}]), Span('osa', [{'id': 5, 'lemma': 'osa', 'upostag': 'S', 'xpostag': 'S', 'feats': OrderedDict([('sg', 'sg'), ('g', 'g')]), 'head': 7, 'deprel': 'nmod', 'deps': '_', 'misc': '_', 'parent_span': <class 'estnltk_core.layer.span.Span'>, 'children': <class 'tuple'>}]), Span('""', [{'id': 6, 'lemma': '""', 'upostag': 'Z', 'xpostag': 'Z', 'feats': OrderedDict(), 'head': 5, 'deprel': 'punct', 'deps': '_', 'misc': '_', 'parent_span': <class 'estnltk_core.layer.span.Span'>, 'children': ()}])]","Text(text='- nende "" mõistliku osa "" eraldumine')"
4,"1 0 root H,2 1 nmod S",Talibani pealetungi,"[Span('Talibani', [{'id': 1, 'lemma': 'Taliban', 'upostag': 'H', 'xpostag': 'H', 'feats': OrderedDict([('sg', 'sg'), ('g', 'g')]), 'head': 0, 'deprel': 'root', 'deps': '_', 'misc': '_', 'parent_span': None, 'children': <class 'tuple'>}]), Span('pealetungi', [{'id': 2, 'lemma': 'pealetung', 'upostag': 'S', 'xpostag': 'S', 'feats': OrderedDict([('sg', 'sg'), ('g', 'g')]), 'head': 1, 'deprel': 'nmod', 'deps': '_', 'misc': '_', 'parent_span': <class 'estnltk_core.layer.span.Span'>, 'children': <class 'tuple'>}])]",Text(text='Talibani pealetungi eest')
...,...,...,...,...
1081,"1 2 nmod H,2 0 root S,3 2 nmod H",Harju maakonnas Sakus,"[Span('Harju', [{'id': 1, 'lemma': 'Harju', 'upostag': 'H', 'xpostag': 'H', 'feats': OrderedDict([('sg', 'sg'), ('g', 'g')]), 'head': 2, 'deprel': 'nmod', 'deps': '_', 'misc': '_', 'parent_span': <clas

In [19]:
# NER/TIMEX/OTHER
combinations = []
for idx, row in matches_df.iterrows():
    combination = []
    for stanza_word in row['phrase_spans']:
        combination.append(get_ner_timex(row['parent_phrase'], stanza_word))
    combinations.append("-".join(combination))

matches_df.insert(0, 'ner/timex/other', combinations)

In [20]:
matches_df

,ner/timex/other,pattern,phrase_text,phrase_spans,parent_phrase
0,OTHER-OTHER,"1 0 root Y,2 1 nmod S",MM turniiriks,"[Span('MM', [{'id': 3, 'lemma': 'mm', 'upostag': 'Y', 'xpostag': 'Y', 'feats': OrderedDict([('?', '?')]), 'head': 0, 'deprel': 'root', 'deps': '_', 'misc': '_', 'parent_span': None, 'children': <class 'tuple'>}]), Span('turniiriks', [{'id': 5, 'lemma': 'turniir', 'upostag': 'S', 'xpostag': 'S', 'feats': OrderedDict([('sg', 'sg'), ('tr', 'tr')]), 'head': 3, 'deprel': 'nmod', 'deps': '_', 'misc': '_', 'parent_span': <class 'estnltk_core.layer.span.Span'>, 'children': <class 'tuple'>}])]",Text(text='veel kümneks MM-turniiriks')
1,OTHER-OTHER-OTHER,"1 2 punct Z,2 0 root S,3 2 punct Z",", häbi .","[Span(',', [{'id': 6, 'lemma': ',', 'upostag': 'Z', 'xpostag': 'Z', 'feats': OrderedDict(), 'head': 10, 'deprel': 'punct', 'deps': '_', 'misc': '_', 'parent_span': <class 'estnltk_core.layer.span.Span'>, 'children': ()}]), Span('häbi', [{'id': 10, 'lemma': 'häbi', 'upostag': 'S', 'xpostag': 'S', 'feats': OrderedDict([('sg', 'sg'), ('n', 'n')]), 'head': 2, 'deprel': 'conj', 'deps': '_', 'misc': '_', 'parent_span': <class 'estnltk_core.layer.span.Span'>, 'children': <class 'tuple'>}]), Span('.', [{'id': 11, 'lemma': '.', 'upostag': 'Z', 'xpostag': 'Z', 'feats': OrderedDict(), 'head': 10, 'deprel': 'punct', 'deps': '_', 'misc': '_', 'parent_span': <class 'estnltk_core.layer.span.Span'>, 'children': ()}])]","Text(text='Sajad politseinikud kahe mehe vastu , no on alles häbi . ""')"
2,OTHER-OTHER-OTHER,"1 2 punct Z,2 0 root S,3 2 punct Z",", häbi .","[Span(',', [{'id': 6, 'lemma': ',', 'upostag': 'Z', 'xpostag': 'Z', 'feats': OrderedDict(), 'head': 10, 'deprel': 'punct', 'deps': '_', 'misc': '_', 'parent_span': <class 'estnltk_core.layer.span.Span'>, 'children': ()}]), Span('häbi', [{'id': 10, 'lemma': 'häbi', 'upostag': 'S', 'xpostag': 'S', 'feats': OrderedDict([('sg', 'sg'), ('n', 'n')]), 'head': 2, 'deprel': 'conj', 'deps': '_', 'misc': '_', 'parent_span': <class 'estnltk_core.layer.span.Span'>, 'children': <class 'tuple'>}]), Span('.', [{'id': 11, 'lemma': '.', 'upostag': 'Z', 'xpostag': 'Z', 'feats': OrderedDict(), 'head': 10, 'deprel': 'punct', 'deps': '_', 'misc': '_', 'parent_span': <class 'estnltk_core.layer.span.Span'>, 'children': ()}])]","Text(text='Sajad politseinikud kahe mehe vastu , no on alles häbi . ""')"
3,OTHER-OTHER-OTHER,"1 2 punct Z,2 0 root S,3 2 punct Z",""" osa ""","[Span('""', [{'id': 3, 'lemma': '""', 'upostag': 'Z', 'xpostag': 'Z', 'feats': OrderedDict(), 'head': 5, 'deprel': 'punct', 'deps': '_', 'misc': '_', 'parent_span': <class 'estnltk_core.layer.span.Span'>, 'children': ()}]), Span('osa', [{'id': 5, 'lemma': 'osa', 'upostag': 'S', 'xpostag': 'S', 'feats': OrderedDict([('sg', 'sg'), ('g', 'g')]), 'head': 7, 'deprel': 'nmod', 'deps': '_', 'misc': '_', 'parent_span': <class 'estnltk_core.layer.span.Span'>, 'children': <class 'tuple'>}]), Span('""', [{'id': 6, 'lemma': '""', 'upostag': 'Z', 'xpostag': 'Z', 'feats': OrderedDict(), 'head': 5, 'deprel': 'punct', 'deps': '_', 'misc': '_', 'parent_span': <class 'estnltk_core.layer.span.Span'>, 'children': ()}])]","Text(text='- nende "" mõistliku osa "" eraldumine')"
4,ORG-OTHER,"1 0 root H,2 1 nmod S",Talibani pealetungi,"[Span('Talibani', [{'id': 1, 'lemma': 'Taliban', 'upostag': 'H', 'xpostag': 'H', 'feats': OrderedDict([('sg', 'sg'), ('g', 'g')]), 'head': 0, 'deprel': 'root', 'deps': '_', 'misc': '_', 'parent_span': None, 'children': <class 'tuple'>}]), Span('pealetungi', [{'id': 2, 'lemma': 'pealetung', 'upostag': 'S', 'xpostag': 'S', 'feats': OrderedDict([('sg', 'sg'), ('g', 'g')]), 'head': 1, 'deprel': 'nmod', 'deps': '_', 'misc': '_', 'parent_span': <class 'estnltk_core.layer.span.Span'>, 'children': <class 'tuple'>}])]",Text(text='Talibani pealetungi eest')
...,...,...,...,...,...
1081,LOC-LOC-OTHER,"1 2 nmod H,2 0 root S,3 2 nmod H",Harju maakonnas Sakus,"[Span('Harju', [{'id': 1, 'lemma': 'Harju', 'upostag': 'H', 'xpostag': 'H', 'feats': Ordere

In [21]:
# adding information about pattern status (with ner/timex) -> 0 (pattern does not exist in pattern list), 
# 1 (pattern exists in pattern list)
pattern_statuses = []
for idx, row in matches_df.iterrows():
    ner_timex = row['ner/timex/other'].split('-')
    pattern_splitted = [part.split() for part in row['pattern'].split(',')]
    for i in range(len(pattern_splitted)):
        pattern_splitted[i].insert(1, ner_timex[i])
    for i in range(len(pattern_splitted)):
        pattern_splitted[i] = " ".join(pattern_splitted[i])
    new_pattern = ",".join(pattern_splitted)
    
    if new_pattern in patterns:
        pattern_statuses.append(1)
    else:
        pattern_statuses.append(0)

matches_df.insert(2, 'pattern_status_ner_timex', pattern_statuses)


In [22]:
# adding column with start and end positions of phrases
phrase_startends = []
for idx, row in matches_df.iterrows():
    start = row['phrase_spans'][0].start
    end = row['phrase_spans'][-1].end
    phrase_startends.append([start, end])
    
matches_df.insert(4, 'startend', phrase_startends)

In [23]:
matches_df_sorted = matches_df.sort_values(by=['ner/timex/other'])

In [24]:
matches_df_sorted

,ner/timex/other,pattern,pattern_status_ner_timex,phrase_text,startend,phrase_spans,parent_phrase
40,LOC-LOC,"1 0 root H,2 1 nmod S",0,Hebroni linna,"[0, 13]","[Span('Hebroni', [{'id': 1, 'lemma': 'Hebroni', 'upostag': 'H', 'xpostag': 'H', 'feats': OrderedDict([('sg', 'sg'), ('g', 'g')]), 'head': 0, 'deprel': 'root', 'deps': '_', 'misc': '_', 'parent_span': None, 'children': <class 'tuple'>}]), Span('linna', [{'id': 2, 'lemma': 'linn', 'upostag': 'S', 'xpostag': 'S', 'feats': OrderedDict([('sg', 'sg'), ('g', 'g')]), 'head': 1, 'deprel': 'nmod', 'deps': '_', 'misc': '_', 'parent_span': <class 'estnltk_core.layer.span.Span'>, 'children': <class 'tuple'>}])]",Text(text='Hebroni linna üle')
1081,LOC-LOC-OTHER,"1 2 nmod H,2 0 root S,3 2 nmod H",0,Harju maakonnas Sakus,"[0, 21]","[Span('Harju', [{'id': 1, 'lemma': 'Harju', 'upostag': 'H', 'xpostag': 'H', 'feats': OrderedDict([('sg', 'sg'), ('g', 'g')]), 'head': 2, 'deprel': 'nmod', 'deps': '_', 'misc': '_', 'parent_span': <class 'estnltk_core.layer.span.Span'>, 'children': ()}]), Span('maakonnas', [{'id': 2, 'lemma': 'maakond', 'upostag': 'S', 'xpostag': 'S', 'feats': OrderedDict([('sg', 'sg'), ('in', 'in')]), 'head': 0, 'deprel': 'root', 'deps': '_', 'misc': '_', 'parent_span': None, 'children': <class 'tuple'>}]), Span('Sakus', [{'id': 3, 'lemma': 'Saku', 'upostag': 'H', 'xpostag': 'H', 'feats': OrderedDict([('sg', 'sg'), ('in', 'in')]), 'head': 2, 'deprel': 'nmod', 'deps': '_', 'misc': '_', 'parent_span': <class 'estnltk_core.layer.span.Span'>, 'children': ()}])]",Text(text='Harju maakonnas Sakus')
542,LOC-OTHER,"1 0 root H,2 1 nmod S",1,Eestist arengus,"[0, 15]","[Span('Eestist', [{'id': 1, 'lemma': 'Eesti', 'upostag': 'H', 'xpostag': 'H', 'feats': OrderedDict([('sg', 'sg'), ('el', 'el')]), 'head': 0, 'deprel': 'root', 'deps': '_', 'misc': '_', 'parent_span': None, 'children': <class 'tuple'>}]), Span('arengus', [{'id': 2, 'lemma': 'areng', 'upostag': 'S', 'xpostag': 'S', 'feats': OrderedDict([('sg', 'sg'), ('in', 'in')]), 'head': 1, 'deprel': 'nmod', 'deps': '_', 'misc': '_', 'parent_span': <class 'estnltk_core.layer.span.Span'>, 'children': ()}])]",Text(text='Eestist arengus')
147,LOC-OTHER,"1 0 root H,2 1 nmod S",1,Inglismaa EM-mängu,"[0, 18]","[Span('Inglismaa', [{'id': 1, 'lemma': 'Inglismaa', 'upostag': 'H', 'xpostag': 'H', 'feats': OrderedDict([('sg', 'sg'), ('g', 'g')]), 'head': 0, 'deprel': 'root', 'deps': '_', 'misc': '_', 'parent_span': None, 'children': <class 'tuple'>}]), Span('EM-mängu', [{'id': 2, 'lemma': 'EM-mäng', 'upostag': 'S', 'xpostag': 'S', 'feats': OrderedDict([('sg', 'sg'), ('g', 'g')]), 'head': 1, 'deprel': 'nmod', 'deps': '_', 'misc': '_', 'parent_span': <class 'estnltk_core.layer.span.Span'>, 'children': <class 'tuple'>}])]",Text(text='Inglismaa EM-mängu tõttu')
20,LOC-OTHER,"1 0 root H,2 1 nmod S",1,Saka rahvamaja,"[0, 14]","[Span('Saka', [{'id': 1, 'lemma': 'Saka', 'upostag': 'H', 'xpostag': 'H', 'feats': OrderedDict([('sg', 'sg'), ('g', 'g')]), 'head': 0, 'deprel': 'root', 'deps': '_', 'misc': '_', 'parent_span': None, 'children': <class 'tuple'>}]), Span('rahvamaja', [{'id': 2, 'lemma': 'rahvamaja', 'upostag': 'S', 'xpostag': 'S', 'feats': OrderedDict([('sg', 'sg'), ('g', 'g')]), 'head': 1, 'deprel': 'nmod', 'deps': '_', 'misc': '_', 'parent_span': <class 'estnltk_core.layer.span.Span'>, 'children': <class 'tuple'>}])]",Text(text='Saka rahvamaja juures')
...,...,...,...,...,...,...,...
402,PER-PER-OTHER,"1 0 root H,2 1 flat S,3 1 flat S",1,Erik Konka wrote,"[25, 41]","[Span('Erik', [{'id': 6, 'lemma': 'Erik', 'upostag': 'H', 'xpostag': 'H', 'feats': OrderedDict([('sg', 'sg'), ('n', 'n')]), 'head': 5, 'deprel': 'appos', 'deps': '_', 'misc': '_', 'parent_span': <class 'estnltk_core.layer.span.Span'>, 'children': <class 'tuple'>}]), Span('Konka', [{'id': 7, 'lemma': 'konka', 'upostag': 'S', 'xpostag': 'S', 'feats': OrderedDict([('sg', 'sg'), ('g', 'g')]), 'head': 6, 'deprel': 'flat', 'deps': '_', 'misc': '_', 'parent_span': <class 'estnltk_core.layer

In [25]:
con = sqlite3.connect("noun_phrases.db")
cur = con.cursor()
cur.execute("CREATE TABLE incorrect_phrase_patterns(ID INTEGER PRIMARY KEY, ner_timex_other TEXT, pattern TEXT, pattern_status_ner_timex INTEGER, phrase_text TEXT, startend TEXT, parent_phrase TEXT)")

In [26]:
con.close()

In [27]:
for idx, row in matches_df_sorted.iterrows():
    print(text_to_json(row['parent_phrase']))
    stri = str(row['startend'])
    print(eval(stri))
    break

{"text": "Hebroni linna üle", "meta": {"document_creation_time": "2024-03-10T01:01", "sentence_id": 4233814, "document_id": 222710, "sentence_startend": [602, 751], "subcorpus": "aja_EPL", "phrase_type": "nmod_phrase", "phrase_start_end": [130, 147]}, "layers": [{"name": "timexes", "attributes": ["tid", "type", "value", "temporal_function", "anchor_time_id", "mod", "quant", "freq", "begin_point", "end_point", "part_of_interval"], "secondary_attributes": [], "parent": null, "enveloping": null, "ambiguous": false, "serialisation_module": null, "meta": {}, "spans": []}, {"name": "tokens", "attributes": [], "secondary_attributes": [], "parent": null, "enveloping": null, "ambiguous": false, "serialisation_module": null, "meta": {}, "spans": [{"base_span": [0, 7], "annotations": [{}]}, {"base_span": [8, 13], "annotations": [{}]}, {"base_span": [14, 17], "annotations": [{}]}]}, {"name": "compound_tokens", "attributes": ["type", "normalized"], "secondary_attributes": [], "parent": null, "envel

In [28]:
con = sqlite3.connect("noun_phrases.db")
cur = con.cursor()

for idx, row in matches_df_sorted.iterrows():
    parent_phrase_json = text_to_json(row['parent_phrase'])
    startend_str = str(row['startend'])
    cur.execute("""INSERT INTO incorrect_phrase_patterns
                            (ner_timex_other, pattern, pattern_status_ner_timex, phrase_text, startend, parent_phrase)
                            VALUES (?, ?, ?, ?, ?, ?);""", (row['ner/timex/other'], row['pattern'], row['pattern_status_ner_timex'], row['phrase_text'], startend_str, parent_phrase_json))
    
    con.commit()

con.close()